In [5]:
import numpy as np
import os
import random

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.contrib import layers, rnn, seq2seq
from tqdm import tqdm

In [3]:
import fasttext as fs

In [4]:
from data_generator import NLUDataGenerator
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple
from tensorflow.contrib.legacy_seq2seq import rnn_decoder

In [6]:
model_fs = fs.load_model("../GoogleNews-vectors-negative300.bin")

Exception: fastText: Cannot load ../GoogleNews-vectors-negative300.bin due to C++ extension failed to allocate the memory

In [48]:
DataGenDev.vocab_size

713

In [49]:
DataGen.vocab_size

822

In [17]:
PAD = 0
EOS = 1

TimeMajor = False
batch_size = 32
encoder_max_time = 64
DataGen = NLUDataGenerator('../data/train/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)

DataGenDev = NLUDataGenerator('../data/dev/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)

vocab_size = len(DataGen.vocab)
input_embedding_size = 128
encoder_hidden_units = 128
# decoder_hidden_units = encoder_hidden_units * 2

In [18]:
x, m1, m2, _lens = next(DataGen)
for x_, m1_, m2_ in zip(x, m1, m2):
    print(DataGen.decode_sentence(x_))
    print(DataGen.decode_acts(m2_))
    print(DataGen.decode_slots(m1_))
    print("-----")

['dontcare', 'restaurant', 'serves']
['B-inform', 'O', 'O']
['B-pricerange', 'O', 'O']
-----
['i', 'want', 'to', 'find', 'a', 'expensive', 'priced', 'restaurant', 'that', 'serves']
['O', 'O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'B-pricerange', 'O', 'O', 'O', 'O']
-----
['okay', 'thats', 'all', 'thank', 'you', 'good', 'bye']
['B-bye', 'B-bye', 'B-bye', 'B-bye', 'B-bye', 'B-bye', 'B-bye']
['B-bye', 'B-bye', 'B-bye', 'B-bye', 'B-bye', 'B-bye', 'B-bye']
-----
['hi', 'im', 'looking', 'for', 'a', 'restaurant', 'in', 'the', 'centre', 'part', 'of', 'town', 'that', 'serves', 'baskaye', 'food']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-inform', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
['hi', 'im', 'looking', 'for', 'a', 'rerestaurant', 'that', 'serves', 'singaporean', 'food']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-inform', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [19]:
slots_number = len(DataGen.slots_encode)
acts_number = len(DataGen.acts_encode)
print("slots_number: ", slots_number)
print("acts_number: ", acts_number)
print("vocab_size: ", DataGen.vocab_size)

slots_number:  27
acts_number:  19
vocab_size:  822


In [21]:
tf.reset_default_graph()

inputs = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask1 = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask2 = tf.placeholder(shape=(None, None), dtype=tf.int32)
lens = tf.placeholder(shape=(None,), dtype=tf.int32)

embeddings = tf.get_variable('embeddings', [vocab_size, input_embedding_size], tf.float32)

encoded = tf.nn.embedding_lookup(embeddings, inputs)
cell = LSTMCell(input_embedding_size)

outputs, _ = tf.nn.dynamic_rnn(cell, encoded, sequence_length=lens, dtype=tf.float32)


seq_mask = tf.to_float(tf.greater(inputs, PAD))
with tf.variable_scope('lm-reg'):
    first_words = tf.slice(inputs, [0, 1], [-1, -1])
    target = tf.pad(first_words, [[0, 0], [0, 1]])
    nwp = layers.fully_connected(outputs, vocab_size, activation_fn=None)
    lm_loss = seq2seq.sequence_loss(nwp, target, seq_mask)

with tf.variable_scope('mask-pred'):
    m1p = layers.fully_connected(outputs, slots_number, activation_fn=None)
    m2p = layers.fully_connected(outputs, acts_number, activation_fn=None)
    
    prediction_m1 = tf.argmax(m1p, axis = 2)
    prediction_m2 = tf.argmax(m2p, axis = 2)
    
    m1_loss = seq2seq.sequence_loss(m1p, mask1, seq_mask)
    m2_loss = seq2seq.sequence_loss(m2p, mask2, seq_mask)
    
    
lr = tf.Variable(0.01, trainable=False)
opt = tf.train.GradientDescentOptimizer(lr)
train_op = opt.minimize(lm_loss + m1_loss + m2_loss)

In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    avg_loss = -np.ones((3, ))
    alpha = 0.9
    
    pbar = tqdm(range(500))
    for step in pbar:
        x, m1, m2, _lens = next(DataGen)
        
        res = sess.run([lm_loss, m1_loss, m2_loss, train_op], {inputs: x, mask1: m1, mask2: m2, lens: _lens})
        res = np.array(res[:-1])
        
        if avg_loss[0] < 0:
            avg_loss = res
        
        avg_loss = alpha * avg_loss + (1-alpha) * res
        
        pbar.set_description(' '.join('{:.2f}'.format(_) for _ in avg_loss))
        
        if(step%100 == 0):
            
            x_dev, m1_dev, m2_dev, _lens_dev = next(DataGenDev)
            print("AVG_loss: ", avg_loss)
            print("step: ", step)
            
            N = 5 # how many examples show to test:
            pred_1, pred_2 = sess.run([prediction_m1, prediction_m2], {inputs: x_dev[:N], 
                                                              mask1: m1_dev[:N], 
                                                              mask2: m2_dev[:N], 
                                                              lens: _lens_dev[:N]})
            
            for x_, m1_, m2_, p1, p2 in zip(x_dev[:N], m1_dev[:N], m2_dev[:N], pred_1, pred_2):
                print("INPUT: ", DataGenDev.decode_sentence(x_))
                print("m2_tar: ", DataGenDev.decode_acts(m2_))
                print("m2_pred: ", DataGenDev.decode_acts(p2))                
                
                print("-----")
                
                print("m1_tar: ", DataGenDev.decode_slots(m1_))
                print("m1_pred: ", DataGenDev.decode_acts(p1))
                
                print("==========================")
            



  0%|          | 0/500 [00:00<?, ?it/s]

AVG_loss:  [ 6.71100521  3.29597974  2.94340706]
step:  0
INPUT:  ['how', 'about', 'the', 'address']
m2_tar:  ['O', 'O', 'O', 'B-request']
m2_pred:  ['I-reqalts', 'I-hello', 'B-bye', 'I-inform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['O', 'O', 'O', 'B-addr']
m1_pred:  ['B-negate', 'I-affirm', 'I-affirm', 'I-affirm', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['north', 'part', 'of', 'town', 'german', 'food']
m2_tar:  ['B-inform', 'O', 'O', 'O', 'B-inform', 'O']
m2_pred:  ['O', 'I-confirm', 'I-bye', 'I-bye', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
-----
m1_tar:  ['B-area', 'O', 'O', 'O', 'B-food', 'O']


KeyError: 19

In [7]:
file_writer = tf.summary.FileWriter('../logdir/', sess.graph)

## Это чисто черновик, где происходит какая-то фигня с bi-LSTM

In [ ]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

encoder_targets = tf.placeholder(shape=(None), dtype=tf.int32, name='encoder_targets')
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [ ]:
encoder_cell_b = LSTMCell(encoder_hidden_units)
encoder_cell_f = LSTMCell(encoder_hidden_units, reuse=None)

In [ ]:
# encoder_cell = LSTMCell(encoder_hidden_units, reuse=tf.get_variable_scope().reuse)
# encoder_cell_f = LSTMCell(encoder_hidden_units)
((encoder_fw_outputs,
encoder_bw_outputs),
(encoder_fw_final_state,
encoder_bw_final_state)) = (tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell_f,
                                cell_bw=encoder_cell_b,
                                inputs=encoder_inputs_embedded,
                                sequence_length=encoder_inputs_length,
                                dtype=tf.float32, time_major=TimeMajor))
    

In [ ]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h)

In [ ]:
from tensorflow.contrib import layers
next_word_pred = layers.fully_connected(encoder_outputs, vocab_size, ...)



mask1_pred = layers.fully_connected(encoder_outputs, mask1_vocab, ...)
mask2_pred = layers.fully_connected(encoder_outputs, mask2_vocab, ...)


In [ ]:
encoder_final_state_h

In [ ]:
W = tf.Variable(tf.random_uniform([encoder_hidden_units*2, vocab_size], -1, 1), dtype=tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [ ]:
logits = tf.add(tf.matmul(encoder_final_state_h, W), b)
# encoder_logits = tf.reshape(encoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [ ]:
logits

In [ ]:
encoder_targets

In [ ]:
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = encoder_targets)